# Query em Excel utilizando LLM 

In [17]:
from langchain.chat_models import ChatOllama
import pandas as pd
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain_experimental.agents import create_pandas_dataframe_agent
import re

In [107]:
df = pd.read_excel('dataset-vendas.xlsx')
df = df[['Numero','Codigo do produto', 'Quantidade']]
df = df.head(10)

In [ ]:
llm = ChatOllama(model="mistral")

In [8]:
# from langchain.chat_models import ChatOllama
# llm = ChatOllama(model="mistral")
# llm.invoke("O que é correlação?")

In [9]:
# resposta = agent.run("Qual foi o total de vendas por mês?")
# print(resposta)

# Opção 1: Custom prompt + pandas interpreter

Esta opção necessita de vários detalhes para a criação da consulta/tratativa pandas correta.

In [99]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

prompt = PromptTemplate(
    input_variables=["pergunta"],
    template="""
Você é um especialista em Python. Um analista de dados fez a seguinte pergunta sobre um DataFrame chamado `df`.
O dataframe tem as colunas: Quantidade, Numero e Codigo do produto. Não invente novas colunas, mas pode fazer tratativas e agrupamentos com as colunas existentes.
Pergunta:

{pergunta}

Gere apenas o código Python (com pandas) necessário para responder. Não escreva explicações.
Como saída, quero somente o código em Python, pois irei roda-lo diretamente em outra função. Não explique nada.
Exemplo de saída: df['Quantidade'].value_counts().idxmax()
"""
)

# llm = ChatOpenAI(model="gpt-4")
llm = ChatOllama(model="mistral")
chain = LLMChain(llm=llm, prompt=prompt)

pergunta = "Qual o produto com maior quantidade?"
codigo_gerado = chain.run(pergunta)
print(codigo_gerado)

# Opcional: execute o código com segurança
local_vars = {"df": df}
# exec(limpar_codigo(codigo_gerado),{},local_vars)


 df['Quantidade'].value_counts().idxmax()


In [100]:
codigo_gerado

" df['Quantidade'].value_counts().idxmax()"

In [102]:
codigo_gerado = re.sub('python','',codigo_gerado)
codigo_gerado = re.sub('```','',codigo_gerado)
codigo_gerado = re.sub('`','',codigo_gerado)
codigo_gerado = re.sub('\n','',codigo_gerado).strip()

In [103]:
codigo_gerado

"df['Quantidade'].value_counts().idxmax()"

In [129]:
resultado = eval(codigo_gerado,{},local_vars) #eval ou exec, o exec não caputa o retorno
resultado

np.int64(85)

Outra alternativa, ao invés de exec ou eval é o Python_REPL do Langchain

# Opção 2:  LangChain Tools personalizados (como funções)

Há mais controle dos cálculos executados.

In [28]:
import pandas as pd
from langchain.agents import Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType

In [32]:
template = """
Você é um assistente que só pode usar as ferramentas listadas abaixo, referenciando elas pelo nome **sem parênteses**.
Use exatamente o nome da ferramenta.

Ferramentas disponíveis:
- soma_por_categoria: Retorna a soma das vendas por categoria

Quando precisar usar uma ferramenta, escreva apenas o nome dela sem parênteses.
Pergunta: {input}
"""

prompt = PromptTemplate(input_variables=["input"], template=template)


# Seu DataFrame
df = pd.DataFrame({
    "Categoria": ["A", "B", "A", "C"],
    "Valor": [10, 20, 15, 5]
})

# def soma_por_categoria():
#     return df.groupby("Categoria")["Valor"].sum().to_string()

def soma_por_categoria(tool_input: str) -> str:
    # O parâmetro tool_input é o texto que o agente passa para a tool. [ignorar]
    return df.groupby("Categoria")["Valor"].sum().to_string()

tools = [
    Tool(name="soma_por_categoria", func=soma_por_categoria, description="Retorna a soma das vendas por categoria"),
]

llm = ChatOllama(model="mistral")

#agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={"prompt": prompt}
)

In [33]:
resposta = agent.run("Qual a soma das vendas por categoria?")

print(resposta)



> Entering new AgentExecutor chain...
 To find the sum of sales by category, I will use the `soma_por_categoria` tool.

Action: soma_por_categoria
Action Input: No input is needed for this function as it calculates the total sales by category from the existing data.
Observation: Categoria
A    25
B    20
C     5
Thought: I now know the final answer.
Final Answer: A soma das vendas por categoria é: A - 25, B - 20 e C - 5. A soma total é 50.

> Finished chain.


In [34]:
resposta

'A soma das vendas por categoria é: A - 25, B - 20 e C - 5. A soma total é 50.'

# Opção 3 - create_pandas_dataframe_agent

Esta opção é mais direta, mas não há muito controle sobre a criação do código python.

In [130]:
df = pd.read_excel('dataset-vendas.xlsx')
df = df[['Numero','Codigo do produto', 'Quantidade']]
df = df.head(10)

In [134]:
df

,Numero,Codigo do produto,Quantidade
0,27192,1022450,68
1,27227,1022450,72
2,27235,1022450,30
3,27250,1022450,26
4,27254,1022450,94
5,27265,1022450,58
6,27266,1022450,54
7,27269,1022450,49
8,27281,1022450,46
9,27283,1022450,22


In [138]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True)

resposta = agent.run("Qual o numero com maior quantidade vendida?")
print(resposta)



> Entering new AgentExecutor chain...
 Thought: I need to find the row with the maximum quantity sold in the dataframe. I can do this by using the `df.loc` function which allows me to select rows based on some condition.
Action: python_repl_ast
Action Input: `print(df.loc[df['Quantidade'].idxmax()])`Numero                 27254
Codigo do produto    1022450
Quantidade                94
Name: 4, dtype: int64
 I now know the final answer. The number with the highest quantity sold is 27254 with a quantity of 94.
Final Answer: The number with the highest quantity sold is 27254 with a quantity of 94.

> Finished chain.
The number with the highest quantity sold is 27254 with a quantity of 94.


In [139]:
resposta

'The number with the highest quantity sold is 27254 with a quantity of 94.'

# Opção 4 - Usando o PythonREPL

https://python.langchain.com/docs/integrations/tools/python/

Solução mais completa, incluir outras tools com as contas já pré-prontas.

In [ ]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_repl.run("print(1+1)")

Importante ter no prompt template:

{input} – a pergunta do usuário

{tools} – lista descritiva das ferramentas disponíveis

{tool_names} – apenas os nomes das ferramentas, separados por vírgula

{agent_scratchpad} – espaço para o agente mostrar seu raciocínio e ações anteriores

In [151]:
df

,Numero,Codigo do produto,Quantidade
0,27192,1022450,68
1,27227,1022450,72
2,27235,1022450,30
3,27250,1022450,26
4,27254,1022450,94
5,27265,1022450,58
6,27266,1022450,54
7,27269,1022450,49
8,27281,1022450,46
9,27283,1022450,22


tool_names = nomes das tools, lista com os nomes definidos, usados em Action: para escolher a tool

tools = texto com nome + descrição, para informar ao modelo sobre as tools disponíveis

In [161]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI # Example LLM

# Assuming df is your loaded Pandas DataFrame
def query_excel_data(query: str) -> str:
    """A Python shell. Use this to execute python commands on the 'df' DataFrame.
    When you want to see the output of a value, you should print it out with `print(...)`."""
    try:
        # This is a simplified example; a real implementation would involve careful parsing
        # and execution within a secure environment.
        exec_globals = {'df': df, 'print': print}
        exec(query, {}, exec_globals)
        return "Command executed successfully." # Or capture and return printed output
    except Exception as e:
        return f"Error executing command: {e}"

#Definição da tool
excel_query_tool = Tool(
    name="Python_REPL",
    func=query_excel_data,
    description="A Python shell. Use this to execute python commands on the 'df' DataFrame. When you want to see the output of a value, you should print it out with `print(...)`."
)

#Posso criar outras tools e incluir tudo nessa lista
tool_names = [excel_query_tool]

# ... (define LLM and prompt template)
from langchain_core.prompts import PromptTemplate


colunas_info = """
O DataFrame `df` tem as seguintes colunas:
- Número (int) : número do produto
- Codigo do produto (int): código de identificação do produto
- Quantidade (inteiro): quantidade vendida
"""


template = f"""Você é um assistente de dados que pode usar ferramentas para responder perguntas sobre um DataFrame do pandas chamado `df`.

O `df` tem as seguintes colunas:
{colunas_info}

Você tem acesso às seguintes ferramentas:

{{tool_names}}

Use o seguinte formato:

Question: a pergunta que você deve responder
Thought: pense sobre o que fazer
Action: a ação a ser tomada, deve ser uma das [{{tools}}]
Action Input: o input para a ação
Observation: o resultado da ação
... (pode repetir Thought/Action/Action Input/Observation quantas vezes quiser)
Thought: agora que sei a resposta
Final Answer: a resposta final à pergunta original

Comece!

Question: {{input}}
{{agent_scratchpad}}"""


prompt = PromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    template=template
)

llm = ChatOllama(model="mistral")

# Create the agent
agent = create_react_agent(llm, tools=tool_names, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [162]:
# Example usage:
agent_executor.invoke({"input": "Qual o código de itentificação com maior quantidade vendida?"})



> Entering new AgentExecutor chain...
 Thought: Para encontrar o código de produto com maior quantidade vendida, primeiro preciso encontrar o maior valor em Quantidade. Em seguida, devo filtrar a linha no DataFrame onde a quantidade seja igual ao máximo encontrado.

Action: Python_REPL
Action Input: `max_quantity = df['Quantidade'].max()`Error executing command: invalid syntax (<string>, line 1) O erro está ocorrendo porque eu não especifiquei que estou trabalhando com a coluna 'Quantidade'. Agora vamos especificar isso e continuaremos.

Action: Python_REPL
Action Input: `max_quantity = df['Quantidade'].max()`Error executing command: invalid syntax (<string>, line 1) O erro está acontecendo porque o Python REPL está esperando uma única linha de código. Vamos dividir o código em duas partes separadas.

Action: Python_REPL (Parte 1)
Action Input: `max_quantity = df['Quantidade'].max()`Python_REPL (Parte 1) is not a valid tool, try one of [Python_REPL]. Oops! It seems I made a mistake. 

KeyboardInterrupt: 

In [ ]:
## # Opção 4 - Usando o PythonREPL

## Separando a construção da query da execução

In [ ]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI  # ou outro LLM de sua escolha

# Seu DataFrame pandas (exemplo)
import pandas as pd
data = {
    'Número': [1, 2, 3],
    'Codigo do produto': [101, 102, 103],
    'Quantidade': [10, 25, 7],
}
df = pd.DataFrame(data)

# Tool 1: gera a query Python que será executada
def generate_query(user_input: str) -> str:
    # Aqui você pode usar LLM para gerar a query, mas para simplificar:
    # Apenas retorna um código Python que responde a pergunta do usuário.
    # Exemplo simples: para "qual produto mais vendido?"
    if "produto mais vendido" in user_input.lower():
        return "df.loc[df['Quantidade'].idxmax(), 'Codigo do produto']"
    return "None"

query_generator_tool = Tool(
    name="QueryGenerator",
    func=generate_query,
    description="Gera uma query Python para analisar o DataFrame df com base na pergunta do usuário."
)

# Tool 2: executa a query Python e retorna o resultado
def execute_query(query: str) -> str:
    try:
        local_vars = {}
        global_vars = {"df": df}
        # executa e armazena o resultado
        result = eval(query, global_vars, local_vars)
        return str(result)
    except Exception as e:
        return f"Erro ao executar query: {e}"

python_executor_tool = Tool(
    name="PythonExecutor",
    func=execute_query,
    description="Executa uma query Python passada como string e retorna o resultado."
)

tools = [query_generator_tool, python_executor_tool]

# Prompt template para o agente ReAct
template = """
Você é um assistente que responde perguntas sobre um DataFrame pandas chamado df.

Você pode usar duas ferramentas:

- QueryGenerator: cria o código Python necessário para responder a pergunta.
- PythonExecutor: executa o código Python gerado e retorna o resultado.

Use o seguinte formato:

Question: {input}
Thought: Devo primeiro gerar a query, depois executar a query.
Action: nome da ferramenta (QueryGenerator ou PythonExecutor)
Action Input: input para a ferramenta
Observation: resultado da ação
... (pode repetir Thought/Action/Action Input/Observation quantas vezes quiser)
Thought: agora sei a resposta
Final Answer: resposta para o usuário

Comece!

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)

# Inicializa o LLM (aqui uso ChatOpenAI, mas pode ser outro)
llm = ChatOllama(model="mistral")

# Cria o agente
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Exemplo de uso
resposta = agent_executor.invoke({"input": "Qual o código do produto mais vendido?"})
print(resposta)


In [ ]:
## Exemplo com mais de uma tool

In [ ]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import pandas as pd

# Exemplo de DataFrame
df = pd.DataFrame({
    'Código do produto': [101, 102, 103],
    'Quantidade': [10, 25, 7],
})

# Tool 1: Gera uma query Python com base na pergunta
def generate_query(user_input: str) -> str:
    if "produto mais vendido" in user_input.lower():
        return "df.loc[df['Quantidade'].idxmax(), 'Código do produto']"
    elif "quantidade total" in user_input.lower():
        return "df['Quantidade'].sum()"
    else:
        return "df.head()"

query_generator_tool = Tool(
    name="QueryGenerator",
    func=generate_query,
    description="Gera uma string de código Python para analisar o DataFrame df com base na pergunta do usuário."
)

# Tool 2: Retorna diretamente uma query pronta para pegar a maior quantidade
def max_quantity_query(_: str) -> str:
    return "df['Quantidade'].max()"

max_quantity_tool = Tool(
    name="MaxQuantityTool",
    func=max_quantity_query,
    description="Retorna a query df['Quantidade'].max(). Use quando o usuário perguntar sobre a maior quantidade vendida."
)

# Tool 3: Executa a string de código retornada por uma das duas tools anteriores
def execute_query(query: str) -> str:
    try:
        result = eval(query, {"df": df})
        return str(result)
    except Exception as e:
        return f"Erro ao executar: {e}"

python_executor_tool = Tool(
    name="PythonExecutor",
    func=execute_query,
    description="Executa a query Python gerada pelas outras tools. Recebe uma string de código e retorna o resultado."
)

tools = [query_generator_tool, max_quantity_tool, python_executor_tool]

# Prompt ReAct customizado para forçar o fluxo de 2 etapas
prompt = PromptTemplate.from_template("""
Você é um assistente que responde perguntas sobre um DataFrame pandas chamado df.

Etapas:
1. Primeiro, escolha entre gerar uma query com QueryGenerator ou usar a query pronta com MaxQuantityTool.
2. Depois, execute essa query usando a ferramenta PythonExecutor.
3. Finalmente, retorne a resposta ao usuário.

Use o seguinte formato:

Question: {input}
Thought: preciso decidir qual ferramenta usar.
Action: nome da ferramenta (QueryGenerator ou MaxQuantityTool)
Action Input: o texto da pergunta
Observation: string com o código Python
Thought: agora vou executar esse código para obter o resultado
Action: PythonExecutor
Action Input: código Python gerado
Observation: resultado da execução
Thought: agora sei a resposta
Final Answer: resposta para o usuário

Comece!

Question: {input}
{agent_scratchpad}
""")

llm = ChatOllama(model="mistral")
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Teste 1 – ativa a Tool direta (MaxQuantityTool)
resposta1 = agent_executor.invoke({"input": "Qual a maior quantidade vendida?"})
print("\nResposta 1:", resposta1)

# Teste 2 – ativa o gerador de query (QueryGenerator)
resposta2 = agent_executor.invoke({"input": "Qual o código do produto mais vendido?"})
print("\nResposta 2:", resposta2)


# Separando com orquestrador + executor

In [163]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Etapa 1: Roteador (decide qual tool usar)
route_prompt = PromptTemplate.from_template("""
Dada a pergunta abaixo, diga se deve usar a ferramenta 'gerar_query' ou 'query_direta':
{input}
Ferramenta:""")

router_chain = LLMChain(llm=llm, prompt=route_prompt)

# Etapa 2: gerar ou escolher a query
def get_query(route: str, user_input: str) -> str:
    if "direta" in route:
        return "df['Quantidade'].max()"
    else:
        # Aqui pode chamar um LLM ou heurística que gera a query
        if "produto mais vendido" in user_input.lower():
            return "df.loc[df['Quantidade'].idxmax(), 'Código do produto']"
        else:
            return "df.head()"

# Etapa 3: executar
def execute_query(query, df):
    return eval(query, {"df": df})

In [164]:

# Exemplo de uso
user_input = "Qual a maior quantidade vendida?"
df = pd.DataFrame({'Código do produto': [101, 102], 'Quantidade': [10, 25]})

route = router_chain.run(input=user_input)
query = get_query(route, user_input)
result = execute_query(query, df)

print("Query:", query)
print("Resultado:", result)


Query: df['Quantidade'].max()
Resultado: 25


In [165]:

# Exemplo de uso
user_input = "Qual o produto mais vendido?"
df = pd.DataFrame({'Código do produto': [101, 102], 'Quantidade': [10, 25]})

route = router_chain.run(input=user_input)
query = get_query(route, user_input)
result = execute_query(query, df)

print("Query:", query)
print("Resultado:", result)


Query: df['Quantidade'].max()
Resultado: 25


# + uma variação

In [168]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.agents.initialize import initialize_agent
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import pandas as pd

# Simula o DataFrame
df = pd.DataFrame({
    'Produto': ['A', 'B', 'C'],
    'Quantidade': [10, 25, 15]
})

# Tool 1: cálculo direto
def max_quantidade_tool(_):
    return df['Quantidade'].max()

# Tool 2: gera uma query com base na pergunta (via LLM)
def gerar_query_llm(pergunta):
    if "produto mais vendido" in pergunta.lower():
        return "df.loc[df['Quantidade'].idxmax(), 'Produto']"
    elif "quantidade total" in pergunta.lower():
        return "df['Quantidade'].sum()"
    else:
        return "df.head()"

# Tool 3: executa o código Python
def executar_query(codigo_python):
    try:
        return str(eval(codigo_python, {"df": df}))
    except Exception as e:
        return f"Erro ao executar: {e}"

# LLM para roteamento

llm = ChatOllama(model="mistral")

# Roteador (decide qual caminho seguir)
roteador_template = PromptTemplate.from_template(
    """Você é um roteador de ferramentas.
Com base na pergunta abaixo, escolha apenas entre as opções:
- 'direto' → se a pergunta se encaixa em um cálculo direto (por exemplo, 'quantidade máxima')
- 'query' → se precisa interpretar e gerar código Python para ser executado depois

Pergunta: {pergunta}
Resposta:"""
)

roteador_chain = LLMChain(llm=llm, prompt=roteador_template)

def agente_com_roteador(pergunta):
    rota = roteador_chain.run(pergunta).strip().lower().replace("'", "").replace('"', '')

    if rota == "direto":
        return max_quantidade_tool(pergunta)

    elif rota == "query":
        query = gerar_query_llm(pergunta)
        return executar_query(query)

    else:
        return f"Rota não reconhecida: '{rota}'"


# 🧪 Teste
pergunta = "Qual foi o produto mais vendido?"
resposta = agente_com_roteador(pergunta)
print(f"Pergunta: {pergunta}\nResposta: {resposta}")


Pergunta: Qual foi o produto mais vendido?
Resposta: 25


# ÚLTIMO TESTE

In [171]:
from langchain.chat_models import ChatOllama

In [186]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
import pandas as pd

# Seu dataframe
df = pd.DataFrame({
    "Produto": ["A", "B", "C"],
    "Quantidade": [10, 25, 15]
})

# LLM
llm = ChatOllama(model="mistral")

# 1. ROTEADOR — decide se é 'query' ou 'direto'
# roteador_prompt = PromptTemplate.from_template("""
# Decida se a pergunta abaixo deve ser respondida com:

# - 'direto': se é uma métrica fixa como máximo, mínimo, soma, etc.
# - 'query': se for preciso gerar um código pandas para obter a resposta.

# Pergunta: {pergunta}
# Resposta (apenas 'query' ou 'direto'):
# """)
# roteador_chain = roteador_prompt | llm | StrOutputParser()

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schema = [ResponseSchema(name="rota", description="Nome da rota que deve ser usada: 'query' ou 'max_tool'")]
parser = StructuredOutputParser.from_response_schemas(schema)

prompt = PromptTemplate(
    template="Classifique a pergunta abaixo e retorne a rota correta.\n\nPergunta: {pergunta}\n\nRotas disponíveis: 'query', 'max_tool'\n\nRetorne no formato JSON.",
    input_variables=["pergunta"]
)

roteador_chain = prompt | llm | parser

# 2. TOOL DIRETA — já embute a lógica de cálculo
def max_quantidade_tool(pergunta: str) -> str:
    max_valor = df["Quantidade"].max()
    return f"A maior quantidade vendida foi {max_valor}"

# 3. TOOL DE GERAÇÃO DE FÓRMULA
gerar_formula_prompt = PromptTemplate.from_template("""
Você é um especialista em pandas. Gere apenas uma linha de código em pandas que responde à pergunta abaixo.
O DataFrame se chama `df` e tem as colunas: Produto (str), Quantidade (int)

Pergunta: {pergunta}

Código em pandas:
""")
gerar_formula_chain = LLMChain(prompt=gerar_formula_prompt, llm=llm)

def gerar_query_llm(pergunta: str) -> str:
    return gerar_formula_chain.invoke({"pergunta": pergunta})["text"].strip()

# 4. EXECUTOR FINAL — executa o código pandas gerado
def executar_query(query: str) -> str:
    try:
        local_vars = {"df": df}
        exec(f"resultado = {query}", {}, local_vars)
        return f"Resposta: {local_vars['resultado']}"
    except Exception as e:
        return f"Erro ao executar a query: {e}"

# 5. AGENTE FINAL — junta tudo
def agente_com_roteador(pergunta):
    rota_dict = roteador_chain.invoke({"pergunta": pergunta})
    rota = rota_dict["rota"].lower().strip()

    if rota == "max_tool":
        resposta = max_quantidade_tool(pergunta)
    elif rota == "query":
        formula = gerar_query_llm(pergunta)
        resposta = executar_query(formula)
    else:
        resposta = f"Rota não reconhecida: '{rota}'"

    print(resposta)


In [187]:
print(agente_com_roteador("Qual o produto mais vendido?"))
# → Resposta: A


OutputParserException: Got invalid return object. Expected key `rota` to be present, but got {'rotas': ['query'], 'resposta': 'A rota correta para responder à pergunta é query.'}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [196]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import pandas as pd

# Seu dataframe
df = pd.DataFrame({
    "Produto": ["A", "B", "C"],
    "Quantidade": [10, 25, 15]
})

# Inicializa o LLM
llm = ChatOllama(model="mistral")

# Define o schema para o output parser do roteador
schemas = [
    ResponseSchema(
        name="rota",
        description="Nome da rota a ser usada, que pode ser 'query' ou 'max_tool'"
    )
]
parser = StructuredOutputParser.from_response_schemas(schemas)

# Prompt do roteador (com instrução clara para retornar só o JSON)
format_instructions = parser.get_format_instructions()
# Escapa as chaves para não causar erro no PromptTemplate
format_instructions_escaped = format_instructions.replace("{", "{{").replace("}", "}}")

roteador_prompt = PromptTemplate(
    input_variables=["pergunta"],
    template=(
        "Classifique a pergunta abaixo e retorne SOMENTE um JSON com a rota correta.\n\n"
        "Pergunta: {pergunta}\n\n"
        "Rotas disponíveis: 'query', 'max_tool'\n\n"
        "Retorne no formato JSON:\n"
        f"{format_instructions_escaped}"
    )
)

# Monta o chain do roteador com o parser
roteador_chain = LLMChain(prompt=roteador_prompt, llm=llm, output_parser=parser)

# Tool 1: cálculo direto (exemplo)
def max_quantidade_tool(pergunta: str) -> str:
    max_valor = df["Quantidade"].max()
    return f"A maior quantidade vendida foi {max_valor}"

# Tool 2: gera código pandas para a pergunta (via LLM)
gerar_formula_prompt = PromptTemplate.from_template("""
Você é um especialista em pandas. Gere apenas uma linha de código pandas que responde à pergunta abaixo.
O DataFrame se chama `df` e tem as colunas: Produto (str), Quantidade (int).

Pergunta: {pergunta}

Código em pandas:
""")
gerar_formula_chain = LLMChain(prompt=gerar_formula_prompt, llm=llm)

def gerar_query_llm(pergunta: str) -> str:
    return gerar_formula_chain.invoke({"pergunta": pergunta})["text"].strip()


import re
#Função pra limpar o output da query gerado
def extrair_codigo_python(texto: str) -> str:
    padrao = r"```python\s*(.*?)```"
    resultado = re.search(padrao, texto, re.DOTALL)
    if resultado:
        return resultado.group(1).strip()
    else:
        # Se não encontrar, tenta pegar o texto todo (pode ajustar se quiser)
        return texto.strip()
    

# Tool 3: executa o código Python no df
def executar_query(query: str) -> str:
    try:
        print(f"Executando query: {query}")  # DEBUG
        codigo_limpo = extrair_codigo_python(query)
        print(codigo_limpo)
        local_vars = {"df": df}
        exec(f"resultado = {codigo_limpo}", {}, local_vars)
        return f"Resposta: {local_vars['resultado']}"
    except Exception as e:
        return f"Erro ao executar a query: {e}"

# Função principal que usa o roteador para decidir a rota
def agente_com_roteador(pergunta: str):
    rota_dict = roteador_chain.invoke({"pergunta": pergunta})
    # rota = rota_dict["rota"].lower().strip()
    rota = rota_dict["text"]["rota"].lower().strip()

    if rota == "max_tool":
        resposta = max_quantidade_tool(pergunta)
    elif rota == "query":
        formula = gerar_query_llm(pergunta)
        resposta = executar_query(formula)
    else:
        resposta = f"Rota não reconhecida: '{rota}'"

    print(resposta)

# Exemplo de uso
if __name__ == "__main__":
    pergunta = "Qual o produto mais vendido?"
    agente_com_roteador(pergunta)


Executando query: Para encontrar o produto mais vendido no DataFrame `df`, você pode usar a função `value_counts()` para contar a quantidade de cada produto e então retornar o primeiro índice, que corresponde ao produto mais vendido.

```python
df['Produto'].value_counts().index[0]
```
df['Produto'].value_counts().index[0]
Resposta: A


In [192]:
rota_dict = roteador_chain.invoke({"pergunta": pergunta})
rota_dict

{'pergunta': 'Qual o produto mais vendido?', 'text': {'rota': 'query'}}

In [195]:
pergunta = "Qual a maior quantidade vendida?"
agente_com_roteador(pergunta)


A maior quantidade vendida foi 25


# Exemplo genérico escolhendo tool:

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import ChatOllama

# Exemplo LLM
llm = ChatOllama(model="mistral")

# Descrição das ferramentas (tools)
tools_descriptions = {
    "max_tool": "Retorna o valor máximo da coluna 'Quantidade'.",
    "query_tool": "Gera código pandas para consultas complexas.",
    "sum_tool": "Calcula a soma de valores em uma coluna específica.",
    # ... imagine até 10 tools aqui
}

# Cria o texto com as descrições para o prompt
tools_info_text = "\n".join([f"- {name}: {desc}" for name, desc in tools_descriptions.items()])

roteador_prompt = PromptTemplate(
    input_variables=["pergunta"],
    template=f"""
Você é um roteador inteligente que recebe uma pergunta e escolhe a melhor ferramenta para respondê-la.

Ferramentas disponíveis:
{tools_info_text}

Pergunta: {{pergunta}}

Qual o nome da ferramenta mais apropriada para responder essa pergunta?
Retorne SOMENTE o nome exato da ferramenta (exemplo: max_tool)
"""
)

roteador_chain = LLMChain(prompt=roteador_prompt, llm=llm)

# Função para obter a tool escolhida
def escolher_tool(pergunta: str) -> str:
    tool_nome = roteador_chain.invoke({"pergunta": pergunta}).strip()
    return tool_nome

# Exemplo de uso
pergunta_exemplo = "Qual o maior valor da coluna Quantidade?"
tool_escolhida = escolher_tool(pergunta_exemplo)
print("Tool escolhida:", tool_escolhida)
